In [2]:
import wfdb
import vfdb_annotate_r as my_vfdb

record_id = '418'
qrs_i = my_vfdb.load_qrs_i(record_id)

qrs_amp = my_vfdb.load_qrs_amp(record_id)
ann = my_vfdb.get_annotation(record_id)
annot_positions = my_vfdb.get_annotation_positions(ann)
annotated_r_peaks = my_vfdb.create_qrs_annotation_mapping(qrs_i, qrs_amp, annot_positions)
annotated_rr_segments = my_vfdb.create_rr_annotation(annotated_r_peaks)

print annotated_rr_segments[:,0].size

4300


In [4]:
print vfdb_annotate_r.VFDB_RECORDS

[418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 602, 605, 607, 609, 610, 611, 612, 614, 615]


In [16]:
import numpy
print vfdb_annotate_r.VFDB_RECORDS
print "\nrecord \t num_of_RR"
for record in vfdb_annotate_r.VFDB_RECORDS:
    filepath = vfdb_annotate_r.QRS_LOCATION + str(record) + '_RR.csv'
    rr = numpy.genfromtxt(filepath, delimiter=',')
    print record, '\t', rr[:,0].size

[418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 602, 605, 607, 609, 610, 611, 612, 614, 615]

record 	 num_of_RR
418 	4300
419 	3905
420 	2923
421 	4525
422 	4072
423 	3537
424 	3795
425 	2128
426 	3774
427 	3894
428 	3265
429 	3613
430 	4695
602 	3307
605 	1921
607 	1847
609 	2214
610 	2166
611 	3947
612 	2489
614 	3329
615 	1592


In [10]:
# Find all different labels in every annotation
import pprint

records = [str(x) for x in my_vfdb.VFDB_RECORDS]
for record in records:
    ann = my_vfdb.get_annotation(record)
    #print record
    #pprint.pprint(set(ann.aux_note))
    labels = set(ann.aux_note)
    print record, labels

418 set(['(N\x00', '(VFL\x00'])
419 set(['(N\x00', '(VFL\x00', '(NOISE\x00'])
420 set(['(N\x00', '(NOISE\x00', '(VT\x00'])
421 set(['(N\x00', '(NOISE\x00', '(VT\x00'])
422 set(['(N\x00', '(NOISE\x00', '(VT\x00', '(VFIB\x00'])
423 set(['(N\x00', '(ASYS\x00', '(NOD\x00', '(NOISE\x00', '(VT\x00', '(AFIB\x00'])
424 set(['(N\x00', '(ASYS\x00', '(NSR\x00', '(VFIB\x00', '(NOD\x00', '(NOISE\x00', '(VFL\x00'])
425 set(['(N\x00', '(NOISE\x00', '(VT\x00', '(B\x00'])
426 set(['(VF\x00', '(N\x00', '(SVTA\x00', '(NOISE\x00', '(VT\x00'])
427 set(['(N\x00', '(ASYS\x00', '(VT\x00'])
428 set(['(BI\x00', '(NOISE\x00', '(VT\x00'])
429 set(['(BI\x00', '(NOISE\x00', '(VT\x00', '(VFL\x00'])
430 set(['(VF\x00', '(HGEA\x00', '(ASYS\x00', '(NOISE\x00', '(BI\x00', '(SBR\x00', '(VER\x00', '(VFL\x00', '(VT\x00'])
602 set(['(N\x00', '(PM\x00', '(ASYS\x00', '(VT\x00', '(NOISE\x00'])
605 set(['(NOISE\x00', '(VT\x00'])
607 set(['(SVTA\x00', '(NOD\x00', '(ASYS\x00', '(VT\x00'])
609 set(['(N\x00', '(HGEA\x00', '(VT\x00'